### Latter Classification Neural Network

Trying to build the best CNN architecture for the images we have regarding to this paper : https://arxiv.org/pdf/1606.02228.pdf

## Library Selections
In order to use state of the art toolset for this research paper we installed CUDA 10 , Pytorch 1.0, Python 3.7 and openCV 3.4 on ubuntu 16.04 with NVDIA 1080 GPU.  https://arxiv.org/pdf/1606.02228.pdf


In [1]:
# Import required libraries for this section
import numpy as np
import cv2

import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from torch import nn

# Use GPU if it's available
from collections import OrderedDict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of data set to use as test
validation_size = 0.5
test_validation_size = 0.4

transform = transforms.Compose([ transforms.CenterCrop(1000), transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(), transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_set = dset.ImageFolder(root="data",transform=transform)
dataloader = torch.utils.data.DataLoader(data_set, batch_size=4,shuffle=True,num_workers=2)

# obtain training indices that will be used for test
num_data = len(data_set)
indices = list(range(num_data))
np.random.shuffle(indices)
split = int(np.floor(test_validation_size * num_data))
train_idx, test_idx = indices[split:], indices[:split]
num_train_data = len(test_idx)
split_validation = int(np.floor(validation_size * num_train_data))
test_idx, validation_idx = test_idx[split_validation:], test_idx[:split_validation]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
validation_sampler  = SubsetRandomSampler(validation_idx)
test_sampler  = SubsetRandomSampler(test_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size,
                                           sampler = train_sampler, num_workers=num_workers)
validation_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                           sampler = test_sampler, num_workers=num_workers)
test_loader  = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                           sampler = test_sampler, num_workers=num_workers)

classes = ('ebrus','suminagashis')

In [3]:
from torch.autograd import Variable
import torch.nn.functional as F

class MultilayerCNN(torch.nn.Module):
  
    #Our batch shape for input x is (3, 224, 224)
    #RELU with batchnorm.
    def __init__(self):
        super(MultilayerCNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(16, 8, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(8 * 7 *7, 2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(-1, 8 * 7 *7)
        x = self.fc(x)
        return x

In [4]:
#apply linear learning rate decay 
def adjust_lr(init_lr, optimizer, epoch, n_epochs):
    lr = init_lr * (1 - (epoch // n_epochs))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [5]:
def createLossAndOptimizer(net, learning_rate=0.001):  
    #Loss function
    loss = torch.nn.CrossEntropyLoss() 
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)   
    return(loss, optimizer)

In [6]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    n_batches = len(train_loader)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        #Create our loss and optimizer functions
        loss, optimizer = createLossAndOptimizer(net, learning_rate)
        learning_rate = adjust_lr(learning_rate, optimizer, epoch, n_epochs)
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in validation_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.item()
        
        total_test_loss = 0
        accuracy = 0
        net.eval()
        with torch.no_grad():
            for inputs, labels in test_loader:
            
                #Wrap tensors in Variables
                inputs, labels = Variable(inputs), Variable(labels)
            
                #Forward pass
                test_outputs = net(inputs)
                test_loss_size = loss(test_outputs, labels)
                total_test_loss += test_loss_size.item()
                
                ps = torch.exp(test_outputs)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                
        net.train()
        
        print("Training loss = {:.2f}".format(total_train_loss / len(train_loader)))
        print("Validation loss = {:.2f}".format(total_val_loss / len(validation_loader)))
        print("Test loss = {:.2f}".format(total_test_loss / len(test_loader)))
        print("Test Accuracy = {:.2f}".format(accuracy / len(test_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [7]:
CNN = MultilayerCNN()
trainNet(CNN, batch_size=128, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 128
epochs= 5
learning_rate= 0.001
Epoch 1, 12% 	 train_loss: 0.90 took: 30.50s
Epoch 1, 25% 	 train_loss: 0.61 took: 29.34s
Epoch 1, 37% 	 train_loss: 0.47 took: 27.29s
Epoch 1, 50% 	 train_loss: 0.46 took: 27.60s
Epoch 1, 62% 	 train_loss: 0.30 took: 27.83s
Epoch 1, 75% 	 train_loss: 0.46 took: 27.59s
Epoch 1, 87% 	 train_loss: 0.46 took: 28.17s
Epoch 1, 100% 	 train_loss: 0.44 took: 27.99s
Training loss = 0.34
Validation loss = 0.24
Test loss = 0.23
Test Accuracy = 0.92
Epoch 2, 12% 	 train_loss: 0.35 took: 26.61s
Epoch 2, 25% 	 train_loss: 0.55 took: 26.24s
Epoch 2, 37% 	 train_loss: 0.42 took: 26.47s
Epoch 2, 50% 	 train_loss: 0.40 took: 26.49s
Epoch 2, 62% 	 train_loss: 0.33 took: 27.59s
Epoch 2, 75% 	 train_loss: 0.31 took: 27.16s
Epoch 2, 87% 	 train_loss: 0.30 took: 27.54s
Epoch 2, 100% 	 train_loss: 0.50 took: 25.96s
Training loss = 0.26
Validation loss = 0.26
Test loss = 0.22
Test Accuracy = 0.92
Epoch 3, 12% 	 train_loss: 0.26 took: 2

In [8]:
torch.save(CNN.state_dict(), 'Multilayer_CNN_128_CenterCrop.pth')

In [9]:
trainNet(CNN, batch_size=256, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 256
epochs= 5
learning_rate= 0.001
Epoch 1, 12% 	 train_loss: 0.19 took: 25.95s
Epoch 1, 25% 	 train_loss: 0.43 took: 27.11s
Epoch 1, 37% 	 train_loss: 0.27 took: 27.21s
Epoch 1, 50% 	 train_loss: 0.33 took: 27.03s
Epoch 1, 62% 	 train_loss: 0.30 took: 27.12s
Epoch 1, 75% 	 train_loss: 0.56 took: 27.34s
Epoch 1, 87% 	 train_loss: 0.27 took: 26.73s
Epoch 1, 100% 	 train_loss: 0.24 took: 26.98s
Training loss = 0.22
Validation loss = 0.18
Test loss = 0.15
Test Accuracy = 0.93
Epoch 2, 12% 	 train_loss: 0.28 took: 27.43s
Epoch 2, 25% 	 train_loss: 0.43 took: 26.59s
Epoch 2, 37% 	 train_loss: 0.47 took: 27.29s
Epoch 2, 50% 	 train_loss: 0.23 took: 27.41s
Epoch 2, 62% 	 train_loss: 0.34 took: 27.02s
Epoch 2, 75% 	 train_loss: 0.23 took: 27.41s
Epoch 2, 87% 	 train_loss: 0.26 took: 27.32s
Epoch 2, 100% 	 train_loss: 0.27 took: 27.56s
Training loss = 0.21
Validation loss = 0.20
Test loss = 0.17
Test Accuracy = 0.93
Epoch 3, 12% 	 train_loss: 0.53 took: 2

In [10]:
torch.save(CNN.state_dict(), 'Multilayer_CNN_256_CenterCrop.pth')